In [1]:
from pymongo import MongoClient
import pandas as pd
import time
import folium
from folium import Choropleth, Circle, Marker, Icon, Map, TileLayer
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
import pandas as pd
import json
import random
from getpass import getpass
import requests
import json
import pandas as pd
from getpass import getpass
import dotenv 
import folium
from folium import Choropleth, Circle, Marker, Icon, Map, TileLayer
from folium.plugins import HeatMap, MarkerCluster
import geopandas as gpd
token = getpass()

In [2]:
df_countries = pd.read_csv(r'../project-III-geospatial-data/data/countries.csv')
df_offices_cord = pd.read_csv(r'../project-III-geospatial-data/data/office_sgp_coordinates.csv')
geo_json = "../project-III-geospatial-data/data/district.json"
with open(geo_json) as geo_file:
    geo_sgp = json.load(geo_file)

In [3]:
#Set connection to mongoDB
def mongo_connection(database,collection):
    client = MongoClient("localhost:27017")
    db = client[f"{database}"]
    client.list_database_names()
    global c 
    c = db.get_collection(f"{collection}")



In [4]:
mongo_connection("Ironhack","companies")

In [5]:
filter_ = {"overview": {"$regex": ".*design*"}}
projection = {"name":1, "offices.latitude":1, "offices.longitude":1, "offices.country_code":1, "_id":0}
new_list = list(c.find(filter_, projection))



In [6]:
lat_long_list = [(d['name'], d['offices'][0]['latitude'], d['offices'][0]['longitude'], d['offices'][0]['country_code']) for d in new_list if d['offices']]

In [7]:

df = pd.DataFrame(lat_long_list, columns=['Name', 'Latitude', 'Longitude', 'Country code'])
df


,Name,Latitude,Longitude,Country code
0,Gizmoz,37.484130,-122.169472,USA
1,Babelgum,53.344104,-6.267494,GBR
2,Cisco,37.408802,-121.953770,USA
3,YouTube,37.627971,-122.426804,USA
4,Intel,37.384949,-121.967610,USA
...,...,...,...,...
1992,SEO Sumo,25.723408,-80.306331,USA
1993,SolarPower Israel,NaN,NaN,ISR
1994,Willdan Group,33.806525,-117.882314,USA
1995,Contemporary Computer Services,40.775055,-73.088140,USA


In [8]:
country_counts = df['Country code'].value_counts()
new_df = pd.DataFrame({'Country code': country_counts.index, 'Count': country_counts.values})
new_df

,Country code,Count
0,USA,1295
1,GBR,162
2,CAN,81
3,IND,67
4,FRA,47
...,...,...
56,BGD,1
57,AUT,1
58,MEX,1
59,LVA,1


In [9]:
merged_df = new_df.merge(df_countries, left_on='Country code', right_on='cca3')


In [10]:

merged_df.drop(columns='place',inplace=True)
merged_df.drop(columns='cca3',inplace=True)
merged_df.drop(columns='cca2',inplace=True)
merged_df.drop(columns='ccn3',inplace=True)
merged_df.drop(columns='region',inplace=True)
merged_df.drop(columns='Rank',inplace=True)
merged_df.drop(columns='landAreaKm',inplace=True)
merged_df.drop(columns='pop2023',inplace=True)
merged_df.drop(columns='densityMi',inplace=True)
merged_df

,Country code,Count,growthRate,area,country,subregion,density
0,USA,1295,0.00505,9372610.0,United States,Northern America,37.1686
1,GBR,162,0.00338,242900.0,United Kingdom,Northern Europe,279.9851
2,CAN,81,0.00850,9984670.0,Canada,Northern America,4.3256
3,IND,67,0.00808,3287590.0,India,"Southern Asia, South Central Asia",480.5033
4,FRA,47,0.00201,551695.0,France,Western Europe,118.2646
5,ISR,31,0.01507,20770.0,Israel,"Western Asia, The Middle East",423.9612
6,AUS,30,0.01000,7692024.0,Australia,Australia and New Zealand,3.4372
7,NLD,24,0.00309,41850.0,Netherlands,Western Europe,523.2640
8,CHN,23,-0.00015,9706961.0,China,Eastern Asia,151.2696
9,DEU,23,-0.00090,357114.0,Germany,Western Europe,238.4002


In [11]:
# Calculate score for each country
merged_df['Count per 100km2'] = merged_df['Count'] / (merged_df['area'] / 1000)
merged_df['score'] = merged_df['Count'] * merged_df['growthRate'] * merged_df['density'] * merged_df['Count per 100km2']

# Sort dataframe by score in descending order
merged_df = merged_df.sort_values('score', ascending=False)

# Print top 10 countries by score
print(merged_df[['country', 'Count','Count per 100km2', 'growthRate', 'density', 'score']].head(5))


           country  Count  Count per 100km2  growthRate    density  \
22       Singapore      6          8.450704     0.00653  8377.0515   
5           Israel     31          1.492537     0.01507   423.9612   
16       Hong Kong      8          7.246377     0.00037  7134.8657   
1   United Kingdom    162          0.666941     0.00338   279.9851   
0    United States   1295          0.138169     0.00505    37.1686   

          score  
22  2773.629953  
5    295.614856  
16   153.037699  
1    102.247838  
0     33.585094  


In [12]:
filter1 = {"overview": {"$regex": ".*design*"}}
filter2 = {"offices.country_code" : "SGP"}
projection = {"name":1, "offices.latitude":1, "offices.longitude":1, "offices.country_code":1, "_id":0}

sgp_list = list(c.find({
            "$and": 
                [filter1, filter2]}, projection))

lat_long_list = [(d['name'], d['offices'][0]['latitude'], d['offices'][0]['longitude'], d['offices'][0]['country_code']) for d in sgp_list if d['offices']]

In [13]:
lat_long_list

[('Cubic Lime', 1.280574, 103.82392, 'SGP'),
 ('Pluit Solutions', 1.366447, 103.85415, 'SGP'),
 ('Jorbb', 1.308655, 103.861929, 'SGP'),
 ('Jorbb', 1.308655, 103.861929, 'SGP'),
 ('Ground labs', None, None, 'SGP'),
 ('eDLink Services Pte Ltd', None, None, 'SGP'),
 ('Niyati Technologies', None, None, 'IND'),
 ('CASON', 47.3752475, 18.9082252, 'HUN'),
 ('Flextronics International', None, None, 'USA'),
 ('Tata Communications', None, None, 'NLD'),
 ('Huge', 40.7028474, -73.9906813, 'USA'),
 ('Lenovo', None, None, 'USA')]

In [14]:

sgp_df = pd.DataFrame(lat_long_list, columns=['Name', 'Latitude', 'Longitude', 'Country code'])
sgp_df = df[df['Country code'] == 'SGP']
sgp_df = sgp_df.drop_duplicates(subset=['Name', 'Latitude', 'Longitude'])
new_values = {'Name': ['Ground labs', 'eDLink Services Pte Ltd'], 
              'Latitude': [1.2956954, 1.2758965], 
              'Longitude': [103.8568923, 103.8127756],
              'Country code': ['SGP', 'SGP']}

new_df = pd.DataFrame(new_values)

df = sgp_df.append(new_df)

df = df.dropna()

df = df.drop_duplicates()

df = df.reset_index(drop=True)

df


C:\Users\Tim_K\AppData\Local\Temp\ipykernel_4992\1764065468.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = sgp_df.append(new_df)


,Name,Latitude,Longitude,Country code
0,Cubic Lime,1.280574,103.823920,SGP
1,Pluit Solutions,1.366447,103.854150,SGP
2,Jorbb,1.308655,103.861929,SGP
3,Ground labs,1.295695,103.856892,SGP
4,eDLink Services Pte Ltd,1.275896,103.812776,SGP


In [15]:
df.drop(columns='Country code',inplace=True)

In [16]:
singapore_map = Map(location = [1.287953, 103.851784], zoom_start = 11)

In [17]:
folium.Choropleth(
    geo_data=geo_sgp,
    data=df,
    columns=df.columns,
    key_on="feature.properties.name",
).add_to(singapore_map)

In [18]:
singapore_map

In [19]:


for index, row in df.iterrows():
            icon = Icon(
            color = "blue",
            prefix = "fa",
            icon = 'suitcase',
            icon_color = "white"
        )
            folium.Marker(location=[row['Latitude'], row['Longitude']], tooltip=row['Name'], icon=icon).add_to(singapore_map)


In [19]:
singapore_map


In [20]:
df = df.drop([0, 2])

In [21]:
for index, row in df.iterrows():
    circle = Circle(location=[row['Latitude'], row['Longitude']], radius=1000, color='red', fill=True, fill_opacity=0.3)
    circle.add_to(singapore_map)
    circle = Circle(location=[row['Latitude'], row['Longitude']], radius=500, color='red', fill=True, fill_opacity=0.3)
    circle.add_to(singapore_map)



In [22]:
singapore_map

In [23]:
# The companies are in 5 #districts: 3,4,7,8 en 20, as district 3&4 and 7&8 are very close to each other we will continue with 3 locations to compare:

# Distruct 3: 1.275896	103.812776
# District 7: 1.295695	103.856892
# District 20: 1.366447	103.854150

In [23]:
list_dict_locations = [
        {"name": "District 20", "lat": "1.366447", "long": "103.854150"},
        {"name": "District 7", "lat": "1.295695", "long": "103.856892"},
        {"name": "District 3", "lat": "1.275896", "long": "103.812776"}
]


In [24]:
def get_foursquare_data():
    base_url = "https://api.foursquare.com/v3/places/search"
    locations = (list_dict_locations)
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    data_frames = []
    for location in locations:
        links = [
            f"query=school&ll={location['lat']}%2C{location['long']}&radius=1000",
            f"ll={location['lat']}%2C{location['long']}&radius=500&chains=ab4c54c0-d68a-012e-5619-003048cad9da",
            f"ll={location['lat']}%2C{location['long']}&radius=1000&categories=10032",
            f"ll={location['lat']}%2C{location['long']}&radius=500&categories=13377",
            f"ll={location['lat']}%2C{location['long']}&radius=10000&categories=18008",
            f"ll={location['lat']}%2C{location['long']}&radius=1000&categories=11134"
        ]
        counts = []
        for link in links:
            url = f"{base_url}?{link}"
            response = requests.get(url, headers=headers)
            count = len(response.json().get("results", []))
            counts.append(count)
        df = pd.DataFrame({"Category": ["Schools 1km", "Starbucks 500m", "Night Clubs 1km", "Vegan Restaurants 500m", "Basketball Courts 10km", "Pet Grooming Services 1km"], "Count": counts})
        df["Location"] = location["name"]
        data_frames.append(df)
    return pd.concat(data_frames).pivot(index="Location", columns="Category", values="Count")

dataframe_best_district = get_foursquare_data()
dataframe_best_district


In [26]:
dataframe_best_district = get_foursquare_data()
dataframe_best_district

In [25]:
def name_coordinates (dict_):
    
    processed_dict = {"name": dict_["name"],
                     "lat": dict_["geocodes"]["main"]["latitude"],
                     "lon": dict_["geocodes"]["main"]["longitude"]}
    
    return processed_dict

In [26]:
singapore_map

In [27]:
def map_public_transport(city):

        # Train station
        url = "https://api.foursquare.com/v3/places/search?ll=1.244463%2C103.834701&radius=25000&categories=19047"

        headers = {
            "accept": "application/json",
            "Authorization": "fsq3wrzINxzk4PynA7gLZ5EdvOrLwygPimqKwxWyEORriC4="
        }

        response = requests.get(url, headers=headers)
        train_list = []
        for i in response.json()["results"]:
            train_list.append(name_coordinates(i))
        
        df_train = pd.DataFrame(train_list)

        for index, row in df_train.iterrows():
            icon = Icon(
            color = "gray",
            fill_opacity = 0.5,
            prefix = "fa",
            icon = 'train',
            icon_color = "white"
        )
            folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=icon).add_to(singapore_map)

        # Airport
        url = "https://api.foursquare.com/v3/places/search?query=Airport&near=Singapore&limit=1"


        headers = {
            "accept": "application/json",
            "Authorization": "fsq3wrzINxzk4PynA7gLZ5EdvOrLwygPimqKwxWyEORriC4="
        }

        response = requests.get(url, headers=headers)
        airport_list = []
        for i in response.json()["results"]:
            airport_list.append(name_coordinates(i))
        
        df_airport = pd.DataFrame(airport_list)

        for index, row in df_airport.iterrows():
            icon = Icon(
            color = "gray",
            fill_opacity = 0.5,
            prefix = "fa",
            icon = 'plane',
            icon_color = "white"
        )
            folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=icon).add_to(singapore_map)
        # Metro
        url = "https://api.foursquare.com/v3/places/search?ll=1.244463%2C103.834701&radius=25000&categories=19046"

        headers = {
            "accept": "application/json",
            "Authorization": "fsq3wrzINxzk4PynA7gLZ5EdvOrLwygPimqKwxWyEORriC4="
        }

        response = requests.get(url, headers=headers)
        metro_list = []
        for i in response.json()["results"]:
            metro_list.append(name_coordinates(i))
        
        df_metro = pd.DataFrame(metro_list)

        for index, row in df_metro.iterrows():
            icon = Icon(
            color = "gray",
            fill_opacity = 0.5,
            prefix = "fa",
            icon = 'subway',
            icon_color = "white"
        )
            folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=icon).add_to(singapore_map)

            # Metro
        url = "https://api.foursquare.com/v3/places/search?query=Train%20station&ll=1.244463%2C103.834701&radius=25000"

        headers = {
            "accept": "application/json",
            "Authorization": "fsq3wrzINxzk4PynA7gLZ5EdvOrLwygPimqKwxWyEORriC4="
        }

        response = requests.get(url, headers=headers)
        tram2_list = []
        for i in response.json()["results"]:
            tram2_list.append(name_coordinates(i))
        
        df_tram2 = pd.DataFrame(tram2_list)

        for index, row in df_tram2.iterrows():
            icon = Icon(
            color = "gray",
            fill_opacity = 0.5,
            prefix = "fa",
            icon = 'train',
            icon_color = "white"
        )
            folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=icon).add_to(singapore_map)

In [28]:
map_public_transport("signapore")

In [29]:
singapore_map

In [30]:
list_dict_locations1 = [
        {"name": "District 20", "lat": "1.366447", "long": "103.854150"}       
]

list_dict_locations2 = [       
        {"name": "District 3", "lat": "1.275896", "long": "103.812776"}
]


list_dict_locations3 = [
        {"name": "District 7", "lat": "1.295695", "long": "103.856892"}
]





In [31]:
def add_places_to_map(locationslist):
    locations = locationslist
    headers = {
        "accept": "application/json",
        "Authorization": token
    }
    for location in locations:
        lat = location["lat"]
        lon = location["long"]

        # Schools 1000m
        url = f"https://api.foursquare.com/v3/places/search?query=school&ll={lat}%2C{lon}&radius=1000"

        headers = {
            "accept": "application/json",
            "Authorization": token 
        }

        response = requests.get(url, headers=headers)

        school_list = []
        for i in response.json()["results"]:
            school_list.append(name_coordinates(i))
        
        df_school = pd.DataFrame(school_list)

        for index, row in df_school.iterrows():
            icon = Icon(
            color = "red",
            prefix = "fa",
            icon = 'book',
            icon_color = "white"
        )
            folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=icon).add_to(singapore_map)
# Starbucks 500m
        url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{lon}&radius=500&chains=ab4c54c0-d68a-012e-5619-003048cad9da"

        headers = {
            "accept": "application/json",
            "Authorization": token
        }

        response = requests.get(url, headers=headers)

        starbucks_list = []
        for i in response.json()["results"]:
            starbucks_list.append(name_coordinates(i))
        
        df_starbucks = pd.DataFrame(starbucks_list)

        for index, row in df_starbucks.iterrows():
            icon = Icon(
            color = "beige",
            prefix = "fa",
            icon = 'coffee',
            icon_color = "white"
        )
            folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=icon).add_to(singapore_map)
# Nightclub 1km
        url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{lon}&radius=1000&categories=10032"

        headers = {
            "accept": "application/json",
            "Authorization": token
        }

        response = requests.get(url, headers=headers)

        nightclub_list = []
        for i in response.json()["results"]:
            nightclub_list.append(name_coordinates(i))
        
        df_nightclub = pd.DataFrame(nightclub_list)

        for index, row in df_nightclub.iterrows():
            icon = Icon(
            color = "purple",
            prefix = "fa",
            icon = 'music',
            icon_color = "white"
        )
            folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=icon).add_to(singapore_map)

        #Vegan restaurant 500m
        url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{lon}&radius=500&categories=13377"

        headers = {
        "accept": "application/json",
        "Authorization": token
        }

        response = requests.get(url, headers=headers)

        vegan_rest_list = []
        for i in response.json()["results"]:
            vegan_rest_list.append(name_coordinates(i))

        vegan_rest_list = []
        for i in response.json()["results"]:
            vegan_rest_list.append(name_coordinates(i))
            
        df_vegan_rest = pd.DataFrame(vegan_rest_list)

        for index, row in df_vegan_rest.iterrows():
            icon = Icon(
            color = "black",
            #opacity = 0.1,
            prefix = "fa", #font-awesome
            icon = 'cutlery',
            icon_color = "white"
        )
            folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=icon).add_to(singapore_map)

        #Basketball 10km
        url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{lon}&radius=10000&categories=18008"

        headers = {
        "accept": "application/json",
        "Authorization": token
        }

        response = requests.get(url, headers=headers)

        basketball_list = []
        for i in response.json()["results"]:
            basketball_list.append(name_coordinates(i))
            
        df_basketball = pd.DataFrame(basketball_list)

        for index, row in df_basketball.iterrows():
            icon = Icon(
            color = "orange",
            #opacity = 0.1,
            prefix = "fa", #font-awesome
            icon = 'futbol',
            icon_color = "white"
        )
            folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=icon).add_to(singapore_map)

        #Groomer 1km
        url = f"https://api.foursquare.com/v3/places/search?ll={lat}%2C{lon}&radius=1000&categories=11134"

        headers = {
        "accept": "application/json",
        "Authorization": token
        }

        response = requests.get(url, headers=headers)

        groomer_list = []
        for i in response.json()["results"]:
            groomer_list.append(name_coordinates(i))
            
        df_groomer = pd.DataFrame(groomer_list)

        for index, row in df_groomer.iterrows():
            icon = Icon(
            color = "green",
            #opacity = 0.1,
            prefix = "fa", #font-awesome
            icon = 'paw',
            icon_color = "white"
        )
            folium.Marker([row['lat'], row['lon']], popup=row['name'], icon=icon).add_to(singapore_map)
        return singapore_map

In [32]:
add_places_to_map(list_dict_locations1)
add_places_to_map(list_dict_locations2)
add_places_to_map(list_dict_locations3)

In [41]:
for index, row in df_offices_cord.iterrows():
            icon = Icon(
            color = "lightblue",
            prefix = "fa",
            icon = 'building',
            icon_color = "black"
        )
            folium.Marker(location=[row['Latitude'], row['Longitude']], popup=[row['Name'], row['Price ($) p/m'], row['Size (m2)'], row['Link']], icon=icon).add_to(singapore_map)



In [42]:
singapore_map

In [ ]:
for index, row in df_offices_cord.iterrows():
            icon = Icon(
            color = "lightblue",
            prefix = "fa",
            icon = 'building',
            icon_color = "black"
        )
            folium.Marker(location=[row['Latitude'], row['Longitude']], popup=[row['Name'], row['Price ($) p/m'], row['Size (m2)'], row['Link']], icon=icon).add_to(singapore_map)

